In [53]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

from sklearn.metrics import accuracy_score

In [2]:
df=pd.read_csv('C:/Users/palt1/Desktop/UFC/df.csv')
df.head(3)

,ID,R_fighter,B_fighter,Winner,date,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,...,R_win_by_KO.TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_total
0,1,Henry Cejudo,Marlon Moraes,Red,2019-06-08,True,Bantamweight,5,0,4,...,2,0,0,8,Orthodox,162.56,162.56,135,32.0,11
1,2,Valentina Shevchenko,Jessica Eye,Red,2019-06-08,True,Women's Flyweight,5,0,3,...,0,2,0,5,Southpaw,165.10,167.64,125,31.0,8
2,3,Tony Ferguson,Donald Cerrone,Red,2019-06-08,False,Lightweight,3,0,3,...,3,6,1,14,Orthodox,180.34,193.04,155,35.0,16


### 범주형 칼럼 확인

In [6]:
list(df.select_dtypes(include=['object', 'bool']))

['R_fighter',
 'B_fighter',
 'Winner',
 'date',
 'title_bout',
 'weight_class',
 'B_Stance',
 'R_Stance']

In [8]:
df=df.drop(['R_fighter','B_fighter','date'], axis=1)
list(df.select_dtypes(include=['object', 'bool']))

['Winner', 'title_bout', 'weight_class', 'B_Stance', 'R_Stance']

### 라벨 인코딩

In [10]:
from sklearn.preprocessing import LabelEncoder

#### 결측값 대체 / stance는 orthodox

In [19]:
df['R_Stance']=df['R_Stance'].fillna('Orthodox')
df['B_Stance']=df['B_Stance'].fillna('Orthodox')

In [56]:
le = LabelEncoder()
df['weight_class'] = le.fit_transform(df['weight_class'])
df['B_Stance'] = le.fit_transform(df['B_Stance'])
df['R_Stance'] = le.fit_transform(df['R_Stance'])
df['title_bout'] = le.fit_transform(df['title_bout'])

In [48]:
df["Winner"] = df["Winner"].replace(["Blue", "Red"], [1,0]).values

# data type : bool -> dbject
df['title_bout']=df['title_bout'].astype('object')

In [34]:
list(df.select_dtypes(include=['object', 'bool']))

['Winner', 'title_bout']

#### 나머지 결측값 평균으로 처리

In [42]:
b_reach = df['B_Reach_cms'].mean()
r_reach = df['R_Reach_cms'].mean()
b_age = df['B_age'].mean()
r_age = df['R_age'].mean()
b_height = df['B_Height_cms'].mean()

In [43]:
df['B_Reach_cms'] = df['B_Reach_cms'].fillna(value=b_reach)
df['R_Reach_cms'] = df['R_Reach_cms'].fillna(value=r_reach)
df['B_age'] = df['B_age'].fillna(value=b_age)
df['R_age'] = df['R_age'].fillna(value=r_age)
df['B_Height_cms'] = df['B_Height_cms'].fillna(value=b_height)

In [44]:
print("Total NaN in dataframe :" , df.isna().sum().sum())
print("Total NaN in each column of the dataframe")
na = []
for index, col in enumerate(df):
    na.append((index, df[col].isna().sum())) 
na_sorted = na.copy()
na_sorted.sort(key = lambda x: x[1], reverse = True) 

for i in range(len(df.columns)):
    print(df.columns[na_sorted[i][0]],":", na_sorted[i][1], "NaN")


Total NaN in dataframe : 0
Total NaN in each column of the dataframe
ID : 0 NaN
Winner : 0 NaN
title_bout : 0 NaN
weight_class : 0 NaN
no_of_rounds : 0 NaN
B_current_lose_streak : 0 NaN
B_current_win_streak : 0 NaN
B_avg_BODY_att : 0 NaN
B_avg_BODY_landed : 0 NaN
B_avg_CLINCH_att : 0 NaN
B_avg_CLINCH_landed : 0 NaN
B_avg_DISTANCE_att : 0 NaN
B_avg_DISTANCE_landed : 0 NaN
B_avg_GROUND_att : 0 NaN
B_avg_GROUND_landed : 0 NaN
B_avg_HEAD_att : 0 NaN
B_avg_HEAD_landed : 0 NaN
B_avg_KD : 0 NaN
B_avg_LEG_att : 0 NaN
B_avg_LEG_landed : 0 NaN
B_avg_PASS : 0 NaN
B_avg_REV : 0 NaN
B_avg_SIG_STR_att : 0 NaN
B_avg_SIG_STR_landed : 0 NaN
B_avg_SIG_STR_pct : 0 NaN
B_avg_SUB_ATT : 0 NaN
B_avg_TD_att : 0 NaN
B_avg_TD_landed : 0 NaN
B_avg_TD_pct : 0 NaN
B_avg_TOTAL_STR_att : 0 NaN
B_avg_TOTAL_STR_landed : 0 NaN
B_longest_win_streak : 0 NaN
B_losses : 0 NaN
B_avg_opp_BODY_att : 0 NaN
B_avg_opp_BODY_landed : 0 NaN
B_avg_opp_CLINCH_att : 0 NaN
B_avg_opp_CLINCH_landed : 0 NaN
B_avg_opp_DISTANCE_att : 0 NaN


In [58]:
df.head()

,ID,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,...,R_win_by_KO.TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_total
0,1,0,1,0,5,0,4,9.200000,6.000000,0.200000,...,2,0,0,8,1,162.56,162.56,135,32.0,11
1,2,0,1,11,5,0,3,14.600000,9.100000,11.800000,...,0,2,0,5,2,165.10,167.64,125,31.0,8
2,3,0,0,6,3,0,3,15.354839,11.322581,6.741935,...,3,6,1,14,1,180.34,193.04,155,35.0,16
3,4,1,0,0,3,0,4,17.000000,14.000000,13.750000,...,1,0,0,6,1,162.56,172.72,135,29.0,9
4,5,1,0,4,3,0,1,17.000000,14.500000,2.500000,...,2,0,0,3,2,187.96,190.50,264,26.0,5


### scaling

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4331 entries, 0 to 4330
Columns: 141 entries, ID to R_total
dtypes: float64(104), int32(1), int64(35), object(1)
memory usage: 4.6+ MB


In [60]:
df_num = df.select_dtypes(include=[np.float, np.int])
scaler = StandardScaler()
df[list(df_num.columns)] = scaler.fit_transform(df[list(df_num.columns)])

In [61]:
df.head()

,ID,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,...,R_win_by_KO.TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_total
0,1,0,3.870601,0,5,0,4,0.040843,-0.041344,-1.062423,...,2,0,0,8,1,-1.895129,-2.052509,135,0.568068,11
1,2,0,3.870601,11,5,0,3,0.791908,0.561818,0.449233,...,0,2,0,5,2,-1.602661,-1.558000,125,0.324739,8
2,3,0,-0.258358,6,3,0,3,0.896896,0.994262,-0.209909,...,3,6,1,14,1,0.152142,0.914547,155,1.298056,16
3,4,1,-0.258358,0,3,0,4,1.125715,1.515203,0.703348,...,1,0,0,6,1,-1.895129,-1.063490,135,-0.161920,9
4,5,1,-0.258358,4,3,0,1,1.125715,1.612487,-0.762698,...,2,0,0,3,2,1.029544,0.667293,264,-0.891907,5
